# Iteration 3
![ToasterOven_3](./../doc/_static/ToasterOven_3.svg)

In [1]:
from miros import ActiveObject
from miros import return_status
from miros import Event
from miros import signals
from miros import spy_on
import time

class ToasterOven(ActiveObject):
  def __init__(self, name):
    super().__init__(name)
    self.history = None

  def light_on(self):
    self.scribble("light_on")

  def light_off(self):
    self.scribble("light_off")

  def heater_on(self):
    self.scribble("heater_on")

  def heater_off(self):
    self.scribble("heater_off")

@spy_on
def door_closed(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.light_off()
    status = return_status.HANDLED
  elif(e.signal == signals.Baking):
    status = oven.trans(baking)
  elif(e.signal == signals.Toasting):
    status = oven.trans(toasting)
  elif(e.signal == signals.INIT_SIGNAL):
    status = oven.trans(off)
  elif(e.signal == signals.Off):
    status = oven.trans(off)
  elif(e.signal == signals.Door_Open):
    status = oven.trans(door_open)
  else:
    oven.temp.fun = oven.top
    status = return_status.SUPER
  return status

@spy_on
def heating(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.heater_on()
    status = return_status.HANDLED
  elif(e.signal == signals.EXIT_SIGNAL):
    oven.heater_off()
    status = return_status.HANDLED
  else:
    oven.temp.fun = door_closed
    status = return_status.SUPER
  return status

@spy_on
def baking(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.scribble("baking")
    oven.history = baking
    status = return_status.HANDLED
  else:
    oven.temp.fun = heating
    status = return_status.SUPER
  return status

@spy_on
def toasting(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.scribble("toasting")
    oven.history = toasting
    status = return_status.HANDLED
  else:
    oven.temp.fun = heating
    status = return_status.SUPER
  return status

@spy_on
def off(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.scribble("off")
    oven.history = off
    status = return_status.HANDLED
  else:
    oven.temp.fun = door_closed
    status = return_status.SUPER
  return status

@spy_on
def door_open(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.light_on()
  elif(e.signal == signals.Door_Close):
    status = oven.trans(oven.history)
  else:
    oven.temp.fun = oven.top
    status = return_status.SUPER
  return status


## Trace output

In [2]:
oven = ToasterOven(name="oven")
oven.live_trace = True
oven.start_at(off)

# toast something
oven.post_fifo(Event(signal=signals.Toasting))
# open the door
oven.post_fifo(Event(signal=signals.Door_Open))
# close the door
oven.post_fifo(Event(signal=signals.Door_Close))

time.sleep(0.01)

[2019-02-03 06:34:05.302441] [oven] e->start_at() top->off
[2019-02-03 06:34:05.308459] [oven] e->Toasting() off->toasting
[2019-02-03 06:34:05.309872] [oven] e->Door_Open() toasting->door_open
[2019-02-03 06:34:05.310633] [oven] e->Door_Close() door_open->toasting


## Spy output

In [3]:
oven = ToasterOven(name="oven")
oven.live_spy = True
oven.live_trace = True
oven.start_at(off)

# open the door
oven.post_fifo(Event(signal=signals.Door_Open))

# close the door
oven.post_fifo(Event(signal=signals.Door_Close))

# Bake something
oven.post_fifo(Event(signal=signals.Baking))

# open the door
oven.post_fifo(Event(signal=signals.Door_Open))

# close the door
oven.post_fifo(Event(signal=signals.Door_Close))

# Toast something
oven.post_fifo(Event(signal=signals.Toasting))

# open the door
oven.post_fifo(Event(signal=signals.Door_Open))

# close the door
oven.post_fifo(Event(signal=signals.Door_Close))


time.sleep(0.1)

[2019-02-03 06:34:05.406966] [oven] e->start_at() top->off
START
SEARCH_FOR_SUPER_SIGNAL:off
SEARCH_FOR_SUPER_SIGNAL:door_closed
ENTRY_SIGNAL:door_closed
light_off
ENTRY_SIGNAL:off
off
INIT_SIGNAL:off
<- Queued:(0) Deferred:(0)
[2019-02-03 06:34:05.410134] [oven] e->Door_Open() off->door_open
Door_Open:off
Door_Open:door_closed
EXIT_SIGNAL:off
SEARCH_FOR_SUPER_SIGNAL:door_open
SEARCH_FOR_SUPER_SIGNAL:door_closed
EXIT_SIGNAL:door_closed
ENTRY_SIGNAL:door_open
light_on
INIT_SIGNAL:door_open
<- Queued:(7) Deferred:(0)
[2019-02-03 06:34:05.412022] [oven] e->Door_Close() door_open->off
Door_Close:door_open
SEARCH_FOR_SUPER_SIGNAL:off
SEARCH_FOR_SUPER_SIGNAL:door_open
SEARCH_FOR_SUPER_SIGNAL:door_closed
EXIT_SIGNAL:door_open
ENTRY_SIGNAL:door_closed
light_off
ENTRY_SIGNAL:off
off
INIT_SIGNAL:off
<- Queued:(6) Deferred:(0)
[2019-02-03 06:34:05.414743] [oven] e->Baking() off->baking
Baking:off
Baking:door_closed
EXIT_SIGNAL:off
SEARCH_FOR_SUPER_SIGNAL:baking
SEARCH_FOR_SUPER_SIGNAL:door_closed

# Iteration 4
![ToasterOven_4](./../doc/_static/ToasterOven_4.svg)

In [73]:
from miros import ActiveObject
from miros import return_status
from miros import Event
from miros import signals
from miros import spy_on
import time

class ToasterOvenMock(ActiveObject):
  def __init__(self, name):
    super().__init__(name)
    self.history = None

  def light_on(self):
    self.scribble("light_on")

  def light_off(self):
    self.scribble("light_off")

  def heater_on(self):
    self.scribble("heater_on")

  def heater_off(self):
    self.scribble("heater_off")
    
  def buzz(self):
    # place your actual buzzer code here,
    # but for now
    # we will scribble into the spy
    self.scribble("buzz")
    
class ToasterOven(ActiveObject):
  def __init__(self, name):
    super().__init__(name)
    self.history = None

  def light_on(self):
    # call to your hardware's light_on driver
    pass

  def light_off(self):
    # call to your hardware's light_off driver
    pass
    
    
  def heater_on(self):
    # call to your hardware's heater on driver
    pass

  def heater_off(self):
    # call to your hardware's heater off driver
     pass
    
  def buzz(self):
    # call to your hardware's buzzer
    pass

@spy_on
def common_features(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.Buzz):
    print("buzz")
    oven.buzz()
    status = return_status.HANDLED
  else:
    oven.temp.fun = oven.top
    status = return_status.SUPER
  return status

@spy_on
def door_closed(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.light_off()
    status = return_status.HANDLED
  elif(e.signal == signals.Baking):
    status = oven.trans(baking)
  elif(e.signal == signals.Toasting):
    status = oven.trans(toasting)
  elif(e.signal == signals.INIT_SIGNAL):
    status = oven.trans(off)
  elif(e.signal == signals.Off):
    status = oven.trans(off)
  elif(e.signal == signals.Door_Open):
    status = oven.trans(door_open)
  else:
    oven.temp.fun = common_features
    status = return_status.SUPER
  return status

@spy_on
def heating(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.heater_on()
    status = return_status.HANDLED
  elif(e.signal == signals.EXIT_SIGNAL):
    oven.heater_off()
    status = return_status.HANDLED
  else:
    oven.temp.fun = door_closed
    status = return_status.SUPER
  return status

@spy_on
def baking(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.history = baking
    status = return_status.HANDLED
  else:
    oven.temp.fun = heating
    status = return_status.SUPER
  return status

@spy_on
def toasting(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.history = toasting
    status = return_status.HANDLED
  else:
    oven.temp.fun = heating
    status = return_status.SUPER
  return status

@spy_on
def off(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.history = off
    status = return_status.HANDLED
  else:
    oven.temp.fun = door_closed
    status = return_status.SUPER
  return status

@spy_on
def door_open(oven, e):
  status = return_status.UNHANDLED
  if(e.signal == signals.ENTRY_SIGNAL):
    oven.light_on()
  elif(e.signal == signals.Door_Close):
    status = oven.trans(oven.history)
  else:
    oven.temp.fun = common_features
    status = return_status.SUPER
  return status


In [70]:
import re
from miros import stripped

def trace_through_all_states():
  oven = ToasterOvenMock(name="oven")
  oven.start_at(door_closed)
  # Open the door
  oven.post_fifo(Event(signal=signals.Door_Open))
  # Close the door
  oven.post_fifo(Event(signal=signals.Door_Close))
  # Bake something
  oven.post_fifo(Event(signal=signals.Baking))
  # Open the door
  oven.post_fifo(Event(signal=signals.Door_Open))
  # Close the door
  oven.post_fifo(Event(signal=signals.Door_Close))
  # Toast something
  oven.post_fifo(Event(signal=signals.Toasting))
  # Open the door
  oven.post_fifo(Event(signal=signals.Door_Open))
  # Close the door
  oven.post_fifo(Event(signal=signals.Door_Close))
  time.sleep(0.01)
  return oven.trace()

def spy_on_light_on():
  oven = ToasterOvenMock(name="oven")
  oven.start_at(door_closed)
  # Open the door to turn on the light
  oven.post_fifo(Event(signal=signals.Door_Open))
  time.sleep(0.01)
  # turn our array into a paragraph
  return "\n".join(oven.spy())

def spy_on_light_off():
  oven = ToasterOvenMock(name="oven")
  # The light should be turned off when we start
  oven.start_at(door_closed)
  time.sleep(0.01)
  # turn our array into a paragraph
  return "\n".join(oven.spy())
  
def spy_on_buzz():
  oven = ToasterOvenMock(name="oven")
  oven.start_at(door_closed)
  # Send the buzz event
  oven.post_fifo(Event(signal=signals.Buzz))
  time.sleep(0.01)
  # turn our array into a paragraph
  return "\n".join(oven.spy())

def spy_on_heater_on():
  oven = ToasterOvenMock(name="oven")
  # The light should be turned off when we start
  oven.start_at(door_closed)
  oven.post_fifo(Event(signal=signals.Toasting))
  time.sleep(0.01)
  # turn our array into a paragraph
  return "\n".join(oven.spy())

def spy_on_heater_off():
  oven = ToasterOvenMock(name="oven")
  # The light should be turned off when we start
  oven.start_at(door_closed)
  oven.post_fifo(Event(signal=signals.Toasting))
  oven.clear_spy()
  oven.post_fifo(Event(signal=signals.Off))
  time.sleep(0.01)
  # turn our array into a paragraph
  return "\n".join(oven.spy())

In [71]:
import re
from miros import stripped

# Confirm our state transitions work as designed
trace_target = """
[2019-02-04 06:37:04.538413] [oven] e->start_at() top->off
[2019-02-04 06:37:04.540290] [oven] e->Door_Open() off->door_open
[2019-02-04 06:37:04.540534] [oven] e->Door_Close() door_open->off
[2019-02-04 06:37:04.540825] [oven] e->Baking() off->baking
[2019-02-04 06:37:04.541109] [oven] e->Door_Open() baking->door_open
[2019-02-04 06:37:04.541393] [oven] e->Door_Close() door_open->baking
[2019-02-04 06:37:04.541751] [oven] e->Toasting() baking->toasting
[2019-02-04 06:37:04.542083] [oven] e->Door_Open() toasting->door_open
[2019-02-04 06:37:04.542346] [oven] e->Door_Close() door_open->toasting
"""

with stripped(trace_target) as stripped_target, \
     stripped(trace_through_all_states()) as stripped_trace_result:
  
  for target, result in zip(stripped_target, stripped_trace_result):
    assert(target == result)

# Confirm our light turns off
assert re.search(r'light_off', spy_on_light_off())

# Confirm our light turns on
assert re.search(r'light_on', spy_on_light_on())

# Confirm the heater turns on
assert re.search(r'heater_on', spy_on_heater_on())

# Confirm the heater turns on
assert re.search(r'heater_off', spy_on_heater_off())

# Confirm our buzzer works
assert re.search(r'buzz', spy_on_buzz())





START
SEARCH_FOR_SUPER_SIGNAL:door_closed
SEARCH_FOR_SUPER_SIGNAL:common_features
ENTRY_SIGNAL:common_features
ENTRY_SIGNAL:door_closed
light_off
INIT_SIGNAL:door_closed
SEARCH_FOR_SUPER_SIGNAL:off
ENTRY_SIGNAL:off
INIT_SIGNAL:off
<- Queued:(0) Deferred:(0)
Toasting:off
Toasting:door_closed
EXIT_SIGNAL:off
SEARCH_FOR_SUPER_SIGNAL:toasting
SEARCH_FOR_SUPER_SIGNAL:door_closed
SEARCH_FOR_SUPER_SIGNAL:heating
ENTRY_SIGNAL:heating
heater_on
ENTRY_SIGNAL:toasting
INIT_SIGNAL:toasting
<- Queued:(0) Deferred:(0)
buzz
